---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    cur_state = None
    sf_list = list()
    with open('university_towns.txt') as f:
        for line in f:
            if '[edit]' in line:
                state = line.split('[')[0]
                cur_state = state
            else:
                region = line.split('(')[0].strip()
                sf_list.append([cur_state,region])
    return pd.DataFrame(sf_list,columns=["State", "RegionName"])
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [57]:
def get_gdp():
    gdp = pd.read_excel('gdplev.xls',
                       skiprows = list(range(0,5))+[6,7] ,parse_cols = "E:G")
    gdp.rename(columns={"Unnamed: 0":"Quarter"}, inplace= True)
    gdp = gdp[gdp['Quarter']>="2000q1"]
    
    for index in gdp.index:
        if index-1 not in gdp.index:
            gdp.loc[index,"GDP_Growth"] = 0
        else:
            gdp.loc[index,"GDP_Growth"] = gdp.loc[index]["GDP in billions of chained 2009 dollars"] - gdp.loc[index-1]["GDP in billions of chained 2009 dollars"]
    return gdp
get_gdp()

,Quarter,GDP in billions of current dollars,GDP in billions of chained 2009 dollars,GDP_Growth
212,2000q1,10031.0,12359.1,0.0
213,2000q2,10278.3,12592.5,233.4
214,2000q3,10357.4,12607.7,15.2
215,2000q4,10472.3,12679.3,71.6
216,2001q1,10508.1,12643.3,-36.0
217,2001q2,10638.4,12710.3,67.0
218,2001q3,10639.5,12670.1,-40.2
219,2001q4,10701.3,12705.3,35.2
220,2002q1,10834.4,12822.3,117.0
221,2002q2,10934.8,12893.0,70.7


In [54]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',
                       skiprows = list(range(0,5))+[6,7] ,parse_cols = "E:G")
    gdp.rename(columns={"Unnamed: 0":"Quarter"}, inplace= True)
    gdp = gdp[gdp['Quarter']>="2000q1"]
    
    for index in gdp.index:
        if index-1 not in gdp.index:
            gdp.loc[index,"GDP_Growth"] = 0
        else:
            gdp.loc[index,"GDP_Growth"] = gdp.loc[index]["GDP in billions of chained 2009 dollars"] - gdp.loc[index-1]["GDP in billions of chained 2009 dollars"]
    gdp = get_gdp()
    r_start_list = list()
    for index in gdp.index[:-1]:
        if gdp.loc[index]['GDP_Growth'] < 0 and gdp.loc[index+1]['GDP_Growth'] < 0:
            return gdp.loc[index]['Quarter']
get_recession_start()

'2008q3'

In [55]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = get_gdp()
    r_start_list = list()
    for index in gdp.index[:-3]:
        if gdp.loc[index]['GDP_Growth'] <0  and gdp.loc[index+1]['GDP_Growth']<0 and \
        gdp.loc[index+2]['GDP_Growth'] >0 and gdp.loc[index+3]['GDP_Growth'] >0:
            return gdp.loc[index+3]['Quarter']
get_recession_end()

'2009q4'

In [56]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = get_gdp()
    r_start_list = list()
    for index in gdp.index[:-3]:
        if gdp.loc[index]['GDP_Growth'] < 0  and gdp.loc[index+1]['GDP_Growth']< 0 and \
        gdp.loc[index+2]['GDP_Growth'] > 0 and gdp.loc[index+3]['GDP_Growth'] > 0:
            return gdp.loc[index+1]['Quarter']
get_recession_bottom()

'2009q2'

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return "ANSWER"

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"